In [1]:
from cloudsafe import *
from utils.folium_utils import get_geojson_grid
database = 'here-api/main.db'
conn = create_connection(database)
departures = select_all_departures(conn)
stations = select_all_stations(conn)
bus_stops = select_all_bus_stops()

In [2]:
coords = stations.iloc[0].coordinates_here.split(',')
m = folium.Map([coords[0], coords[1]], zoom_start=10)
stations[['latitude', 'longitude']] = stations.apply(get_coords, axis=1, result_type="expand")

In [3]:
frequencies = stations.name_overpass.value_counts()

In [4]:
frequencies

Vigerslevvej                     9
Trianglen St.                    7
Strandboulevarden                7
Universitetsparken               7
Baltikavej                       6
                                ..
Teglholmsgade (Teglholm Allé)    1
Gartnerivej                      1
Rumæniensgade                    1
Grækenlandsvej                   1
Ravnsborggade                    1
Name: name_overpass, Length: 458, dtype: int64

In [5]:
rows = []
for station in frequencies.keys():
    first = stations.loc[stations['name_overpass'] == station].iloc[0]
    rows.append({'station_name': station,
                 'longitude': first.longitude,
                 'latitude': first.latitude,
                 'bus_count': frequencies.get(station)})

final = pd.DataFrame(rows)
final['longitude'] = final['longitude'].astype(float)
final['latitude'] = final['latitude'].astype(float)
top_right = [final['latitude'].max(), final['longitude'].max()]
bottom_left = [final['latitude'].min(), final['longitude'].min()]

In [6]:
first

id                                                22
id_overpass                           node/393190127
name_overpass                          Ravnsborggade
coordinates_overpass             55.687494,12.561732
id_here                                    205078942
name_here               Ravnsborggade (Nørrebrogade)
coordinates_here                 55.687535,12.561576
no_data                                            0
duplicate                                          0
latitude                                   55.687535
longitude                                  12.561576
Name: 21, dtype: object

In [7]:
rows

[{'station_name': 'Vigerslevvej',
  'longitude': '12.491016',
  'latitude': '55.670034',
  'bus_count': 9},
 {'station_name': 'Trianglen St.',
  'longitude': '12.577824',
  'latitude': '55.699336',
  'bus_count': 7},
 {'station_name': 'Strandboulevarden',
  'longitude': '12.588667',
  'latitude': '55.703497',
  'bus_count': 7},
 {'station_name': 'Universitetsparken',
  'longitude': '12.558402',
  'latitude': '55.702868',
  'bus_count': 7},
 {'station_name': 'Baltikavej',
  'longitude': '12.597485',
  'latitude': '55.71696',
  'bus_count': 6},
 {'station_name': 'Bellahøj',
  'longitude': '12.51633',
  'latitude': '55.705484',
  'bus_count': 6},
 {'station_name': 'Toftegårds Plads',
  'longitude': '12.516626',
  'latitude': '55.66219',
  'bus_count': 6},
 {'station_name': 'Folehaven',
  'longitude': '12.489274',
  'latitude': '55.652622',
  'bus_count': 6},
 {'station_name': 'Emdrup Torv',
  'longitude': '12.52994',
  'latitude': '55.723076',
  'bus_count': 6},
 {'station_name': 'Sundbyv

In [8]:
final

,station_name,longitude,latitude,bus_count
0,Vigerslevvej,12.491016,55.670034,9
1,Trianglen St.,12.577824,55.699336,7
2,Strandboulevarden,12.588667,55.703497,7
3,Universitetsparken,12.558402,55.702868,7
4,Baltikavej,12.597485,55.716960,6
...,...,...,...,...
453,Teglholmsgade (Teglholm Allé),NaN,NaN,1
454,Gartnerivej,12.559391,55.720553,1
455,Rumæniensgade,12.613443,55.660391,1
456,Grækenlandsvej,12.614795,55.653832,1


In [9]:
# Generating a grid heatmap (utils/folium_utils.py)
grid = get_geojson_grid(top_right, bottom_left)

In [10]:
grid

[{'type': 'FeatureCollection',
  'properties': {'lower_left': [12.45777, 55.61798],
   'upper_right': [12.486202, 55.636300000000006]},
  'features': [{'type': 'Feature',
    'geometry': {'type': 'Polygon',
     'coordinates': [[[12.45777, 55.636300000000006],
       [12.486202, 55.636300000000006],
       [12.486202, 55.61798],
       [12.45777, 55.61798],
       [12.45777, 55.636300000000006]]]}}]},
 {'type': 'FeatureCollection',
  'properties': {'lower_left': [12.486202, 55.61798],
   'upper_right': [12.514634000000001, 55.636300000000006]},
  'features': [{'type': 'Feature',
    'geometry': {'type': 'Polygon',
     'coordinates': [[[12.486202, 55.636300000000006],
       [12.514634000000001, 55.636300000000006],
       [12.514634000000001, 55.61798],
       [12.486202, 55.61798],
       [12.486202, 55.636300000000006]]]}}]},
 {'type': 'FeatureCollection',
  'properties': {'lower_left': [12.514634, 55.61798],
   'upper_right': [12.543066, 55.636300000000006]},
  'features': [{'type'

In [11]:
# Frequencies by latitude and longitude
counts = []
for box in grid:
    upper_right = box["properties"]["upper_right"]
    lower_left = box["properties"]["lower_left"]
    items = final.loc[(final['latitude'] <= upper_right[1]) &
                      (final['longitude'] <= upper_right[0]) &
                      (final['longitude'] >= lower_left[0]) &
                      (final['latitude'] >= lower_left[1])]
    counts.append(len(items))
max_count = max(counts)

In [12]:
# Add GeoJson to map
for i, geo_json in enumerate(grid):
    color = plt.cm.Reds(counts[i] / max_count)
    color = mpl.colors.to_hex(color)
    gj = folium.GeoJson(geo_json,
                        style_function=lambda feature, 
                        color=color: {
                            'fillColor': color,
                            'color': "black",
                            'weight': 2,
                            'dashArray': '5, 5',
                            'fillOpacity': 0.55,
                        })

    m.add_child(gj)

In [13]:
m